In [1]:
# !pip3 install matplotlib
# !pip3 install seaborn
# !pip3 install -U scikit-learn
# !pip3 install tensorflow
# !pip3 install --upgrade tensorflow

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [3]:
# from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

LOAD DATASET : KAGGLE

In [6]:
# data_dir = 'C:/Users/aditi/OneDrive/Desktop/DESKTOP/VS_LANG/Python/LEGALITY_ML/DATASETS/BHSig260-Bengali/BHSig260-Bengali'
data_dir = 'C:/Users/aditi/OneDrive/Desktop/DESKTOP/VS_LANG/Python/LEGALITY_ML/DATASETS/archive/sign_data/sign_data'
# data_dir = 'C:/Users/aditi/OneDrive/Desktop/DESKTOP/VS_LANG/Python/LEGALITY_ML/DATASETS/data/data'

img_width, img_height = 256, 256

datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=40,
                             width_shift_range=0.4,
                             height_shift_range=0.4,
                             shear_range=0.4,
                             zoom_range=0.4,
                             horizontal_flip=True,
                             vertical_flip=True,
                             validation_split=0.2)

train_data = datagen.flow_from_directory(data_dir,
                                         target_size=(img_width, img_height),
                                         batch_size=64,
                                         class_mode='binary',
                                         subset='training')

validation_data = datagen.flow_from_directory(data_dir,
                                            target_size=(img_width, img_height),
                                            batch_size=64,
                                            class_mode='binary',
                                            subset='validation',
                                            # data_format='channels_last',
                                            save_to_dir='validation_data_aug',
                                            save_prefix='img',
                                            save_format='jpeg',
                                            # save_percent_threshold=0.5
                                            )

def lr_scheduler(epoch, lr):
    if epoch % 50 == 0 and epoch != 0:
        lr = lr * 0.1
    return lr

Found 1720 images belonging to 2 classes.
Found 429 images belonging to 2 classes.


CNN ARCHITECTURE

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

C:\Users\aditi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


CNN MODEL COMPILATION

In [8]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
            #   optimizer='rmsprop',
              metrics=['accuracy', 'precision', 'recall'])

CNN MODEL TRAINING

In [9]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='min', min_lr=0.001)
lr_schedule = LearningRateScheduler(lr_scheduler, verbose=1)

history = model.fit(train_data,
                    epochs=200,
                    validation_data=validation_data,
                    callbacks=[es, rlr, lr_schedule])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/200


C:\Users\aditi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/27 ━━━━━━━━━━━━━━━━━━━━ 39:12 157s/step - accuracy: 0.6221 - loss: 0.8566 - precision: 0.7305 - recall: 0.7648

KeyboardInterrupt: 

CNN MODEL EVALUATION

In [ ]:
y_pred = model.predict(validation_data)
y_pred = (y_pred > 0.5)

In [ ]:
scores = model.evaluate(validation_data, verbose=1)
f1 = f1_score(validation_data.classes, y_pred)
print('Accuracy:', scores[1])
print('Precision:', scores[2])
print('Recall:', scores[3])  
print('F1 Score:', f1)                       

CLASSIFICATION REPORT & CONFUSION MATRIX - CNN

In [ ]:
cm = confusion_matrix(validation_data.classes, y_pred)
cr = classification_report(validation_data.classes, y_pred)

In [ ]:
print(cr)
print(cm)

TRAINING HISTORY PLOT

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()